In [1]:
import torch
import random
import numpy as np
import torch.nn as torchNN
import torch.nn.functional as torchF
import matplotlib.pyplot as plt
import gymnasium as gym

from IPython import display
from collections import namedtuple, deque

%matplotlib inline

In [2]:
Transition = namedtuple('Transition', ('nowState', 'action', 'nextState', 'reward'))
class MemoryPool():
    def __init__(self, capacity) -> None:
        self.memory = deque([], maxlen = capacity)
    
    def AddMemory(self, *arg):
        self.memory.append(Transition(*arg))
    
    def Sample(self, BATCHSIZE):
        return random.sample(self.memory, BATCHSIZE)

    def __len__(self):
        return len(self.memory)

In [4]:
# Add network policy and target
class Network(torchNN.Module):
    def __init__(self, stateNum, actionNum, hiddenUnits) -> None:
        super().__init__()
        self.liner = torchNN.Sequential(
            torchNN.Linear(stateNum, hiddenUnits),
            torchNN.LeakyReLU(),
            torchNN.Linear(hiddenUnits, hiddenUnits),
            torchNN.LeakyReLU(),
            torchNN.Linear(hiddenUnits, actionNum),
        )

    def forward(self, nowState):
        return self.liner(nowState)

In [ ]:
class DQNModel():
    def __init__(
        self,
        stateNum,
        actionNum,
        hiddenUnits,
        episold,
        poolCapacity,
        environment,
        learningRate,
        batchSize,
    ):
        '''
        Param:
            stateNum: 
            actionNum:
            hiddenUnits:
            episold:
            poolCapacity:
            environment:
            learningRate:
        '''

        self.policyNetwork = Network(stateNum, actionNum, hiddenUnits)
        self.targetNetwork = Network(stateNum, actionNum, hiddenUnits)
        self.episold = episold
        self.memoryPool = MemoryPool(poolCapacity)
        self.environment = environment
        self.optimizer = torch.optim.Adam(self.policyNetwork.get_parameter(), lr = learningRate)
        self.lossFunc = torchNN.SmoothL1Loss()                      # use L1 lossfunction
        self.batchSize = batchSize

    def ChooseAction(self, nowState) -> torch.Tensor:
        '''
        Func:
            return the next action agent may take
        Param:
            nowState:   the state agent is in
        Return:
            shape:      (1, 1), the first d is used for 'BatchSize'      
        '''
        if random.random() < self.episold:
            return torch.Tensor([[self.environment.action_space.sample()]])
        with torch.no_grad():
            return self.policyNetwork(nowState).max(1).indices.view(1, 1)   
        # TODO: Learn the network's max function and indices function

    def OptimizeNetwork(self):
        if len(self.memoryPool) < self.batchSize:
            return

        data = self.memoryPool.Sample(self.batchSize)
        data = Transition(*zip(*data))

        


In [ ]:
env = gym.make("CartPole-v1")
